In [4]:
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.stats import chisquare, chi2_contingency, ttest_ind, contingency, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# Loading data

### Reading pickle file - can skip loading raw file below

In [5]:
gp_events_df = pd.read_pickle('./data/extracted_raw_gp_events.pkl')

### Read & save raw file

In [3]:
# Read GP data

gp_events_df = pd.read_table(
    '/mnt/shared_folders/eResearch_glaucoma_project/UKBB_Data_healthrelatedoutcomes/gp_clinical.txt', 
    low_memory=True, 
    usecols=['eid', 'event_dt', 'read_2', 'read_3'],
    parse_dates = ['event_dt'],
    date_format='%d/%m/%Y',
)

In [9]:
# Save GP data to pickle

gp_events_df.to_pickle('./data/extracted_raw_gp_events.pkl', compression=None)

### Read UKB codings

In [6]:
# Read ICD10 codes for GP
# Use the Excel file they gave us, not any exported .csv -> getting weird issues with '.' character

read_v2_icd10_csv = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_v2_icd10', usecols=['read_code', 'icd10_code'])[:36664]
read_v3_icd10_csv = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_ctv3_icd10', usecols=['read_code', 'icd10_code'])[:116374]

In [7]:
read_v2_icd9_csv = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_v2_icd9', usecols=['read_code', 'icd9_code'])[:35651]
read_v3_icd9_csv = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_ctv3_icd9', usecols=['read_code', 'icd9_code'])[:67130]

In [8]:
read_v2_descriptions = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_v2_lkp', usecols=['read_code', 'term_description'])[:101881]
read_v3_descriptions = pd.read_excel('./data/coding/primary_care/all_lkps_maps_v4.xlsx', sheet_name='read_ctv3_lkp', usecols=['read_code', 'term_description'])[:419096]

# Functions

In [9]:
def get_matched_gp_codes(icd10_list, icd9_list):
    #icd10_read_v2 = read_v2_icd10_csv[read_v2_icd10_csv['icd10_code'].astype(str).eq('|'.join(icd10_list))]
    #icd10_read_v3 = read_v3_icd10_csv[read_v3_icd10_csv['icd10_code'].astype(str).eq('|'.join(icd10_list))]
    # icd9_read_v2 = read_v2_icd9_csv[read_v2_icd9_csv['icd9_code'].astype(str).eq('|'.join(icd9_list))]
    # icd9_read_v3 = read_v3_icd9_csv[read_v3_icd9_csv['icd9_code'].astype(str).eq('|'.join(icd9_list))]

    icd10_read_v2 = read_v2_icd10_csv[read_v2_icd10_csv['icd10_code'].str.contains('|'.join(icd10_list))]
    icd10_read_v3 = read_v3_icd10_csv[read_v3_icd10_csv['icd10_code'].str.contains('|'.join(icd10_list))]
    
    icd9_read_v2 = read_v2_icd9_csv[read_v2_icd9_csv['icd9_code'].str.contains('|'.join(icd9_list))]
    icd9_read_v3 = read_v3_icd9_csv[read_v3_icd9_csv['icd9_code'].str.contains('|'.join(icd9_list))]


    return icd10_read_v2, icd10_read_v3, icd9_read_v2, icd9_read_v3

In [10]:
def get_read_list(read_category, icd10_df, icd9_df):
    df = pd.DataFrame(columns=['read_category', 'read_code', 'ICD10', 'ICD9', 'description(s)'])
    for i, row in icd10_df.iterrows():
        equivalent_icd9 = pd.NA
        if row['read_code'] in icd9_df['read_code'].values:
            equivalent_icd9 = ', '.join(icd9_df[icd9_df['read_code'] == row['read_code']]['icd9_code'].astype('string').dropna().values)

        descriptions = pd.NA
        if read_category == 'v2':
            descriptions = '; '.join(read_v2_descriptions[read_v2_descriptions['read_code'] == row['read_code']]['term_description'].astype('string').dropna().values)
        elif read_category == 'v3':
            descriptions = '; '.join(read_v3_descriptions[read_v3_descriptions['read_code'] == row['read_code']]['term_description'].astype('string').dropna().values)
        df.loc[len(df)] = [read_category, row['read_code'], row['icd10_code'], equivalent_icd9, descriptions]
    for i, row in icd9_df.iterrows():
        if row['read_code'] in icd10_df['read_code'].values:
            continue
        descriptions = pd.NA
        if read_category == 'v2':
            descriptions = '; '.join(read_v2_descriptions[read_v2_descriptions['read_code'] == row['read_code']]['term_description'].astype('string').dropna().values)
        elif read_category == 'v3':
            descriptions = '; '.join(read_v3_descriptions[read_v3_descriptions['read_code'] == row['read_code']]['term_description'].astype('string').dropna().values)
        df.loc[len(df)] = [read_category, row['read_code'], pd.NA, row['icd9_code'], descriptions]
    return df

def output_gp_codes(icd10_read_v2, icd9_read_v2, icd10_read_v3, icd9_read_v3, export_dir=None):
    df = pd.DataFrame(columns=['read_v2', 'read_v3', 'ICD10', 'ICD9'])

    # Read v2
    v2_df = get_read_list('v2', icd10_read_v2, icd9_read_v2)

    # Read v3
    v3_df = get_read_list('v3', icd10_read_v3, icd9_read_v3)

    df = pd.concat((v2_df, v3_df))

    if export_dir != None:
        df.to_csv(export_dir, header=True, index=False)
    
    return df

In [28]:
def output_automatic_GP_mapping(icd10_list, icd9_list, export_dir=None):
    icd10_read_v2, icd10_read_v3, icd9_read_v2, icd9_read_v3 = get_matched_gp_codes(icd10_list, icd9_list)
    df = output_gp_codes(icd10_read_v2, icd9_read_v2, icd10_read_v3, icd9_read_v3, export_dir)

    all_read_v2 = np.unique(np.concatenate((icd10_read_v2['read_code'].values, icd9_read_v2['read_code'].values)))
    all_read_v3 = np.unique(np.concatenate((icd10_read_v3['read_code'].values, icd9_read_v3['read_code'].values)))

    print(
        f'ICD10, read v2: {', '.join(icd10_read_v2['read_code'].values)}'
        + f'\nICD9, read v2: {', '.join(icd9_read_v2['read_code'].values)}'
        + f'\nICD10, read v3: {', '.join(icd10_read_v3['read_code'].values)}'
        + f'\nICD9, read v3: {', '.join(icd9_read_v3['read_code'].values)}'

        +f'\n\nAll read v2: {', '.join(all_read_v2)}'
        +f'\nAll read v3: {', '.join(all_read_v3)}'
    )
    
    return df, all_read_v2, all_read_v3

# Automated GP mapping for select diseases

### Sleep apnoea

- ICD10: G473
- ICD9: 3272

In [20]:
# Sleep apnoea

output_automatic_GP_mapping(
    icd10_list = ['G473'],
    icd9_list = ['3272'],
    export_dir = './data/coding/primary_care/mapped_sleep_apnoea_primary_care.csv'
)

ICD10, read v2: Fy03., Fy04., H5B.., H5B0., R0051, R0053
ICD9, read v2: 
ICD10, read v3: Fy03., R0051, X0083, X0084, X0085, X0086, X0087, X008F, X76Gw, X76Hk, XE2nU, XM08E
ICD9, read v3: 


,read_category,read_code,ICD10,ICD9,description(s)
0,v2,Fy03.,G473,<NA>,Sleep apnoea; Obstructive sleep apnoea
1,v2,Fy04.,G473,<NA>,Sleep-related respiratory failure; Ondine's curse
2,v2,H5B..,G473,<NA>,Sleep apnoea
3,v2,H5B0.,G473,<NA>,Obstructive sleep apnoea
4,v2,R0051,G470+G473,<NA>,[D]Insomnia with sleep apnoea
5,v2,R0053,G471+G473,<NA>,[D]Hypersomnia with sleep apnoea; [D]Sleep apn...
0,v3,Fy03.,G473,<NA>,Sleep apnoea (& [obstructive]); Sleep apnoea; ...
1,v3,R0051,G473,<NA>,[D]Insomnia with sleep apnoea
2,v3,X0083,G473,<NA>,Sleep apnoea; Sleep hypopnoea; Sleep apnoea sy...
3,v3,X0084,G473,<NA>,Obstructive sleep apnoea; OSA - Obstructive sl...


### H. pylori

- ICD10: B980
- ICD9: 04186 (041.86)

In [21]:
# H. pylori

output_automatic_GP_mapping(
    icd10_list = ['B980'],
    icd9_list = ['04186'],
    export_dir = './data/coding/primary_care/mapped_h_pylori_primary_care.csv'
)

ICD10, read v2: A3BXJ, J1544
ICD9, read v2: 
ICD10, read v3: X301R, XaBE5, XaDsy
ICD9, read v3: 


,read_category,read_code,ICD10,ICD9,description(s)
0,v2,A3BXJ,B980,<NA>,Helicobacter pylori as the cause of diseases c...
1,v2,J1544,K296+B980,<NA>,Helicobacter gastritis
0,v3,X301R,B980,<NA>,Helicobacter gastritis; Helicobacter pylori-as...
1,v3,XaBE5,B980,<NA>,Helicobacter pylori gastrointestinal tract inf...
2,v3,XaDsy,B980,<NA>,Helicobacter pylori as the cause of diseases c...


### Sjogren syndrome

- ICD10: M350
- ICD9: 7102 (710.2)

In [22]:
# Sjogren

output_automatic_GP_mapping(
    icd10_list = ['M350'],
    icd9_list = ['7102'],
    export_dir = './data/coding/primary_care/mapped_sjogren_syndrome_primary_care.csv'
)

ICD10, read v2: F3967, F4A32, H57y3, N002.
ICD9, read v2: F3967, H57y3, N002.
ICD10, read v3: 1927., F3967, F3967, F4A32, F4A32, H57y3, H57y3, N002., X705A, X705B, X705B, X705C, X705D, X705E, X705E, X705F, XE1DG, XE1DG, XE1DG, Xa0hF, Xa0hF, Xa0hF, Xa9BV, Xa9BV
ICD9, read v3: F3967, H57y3, X705A, X705B, X705C, X705D, X705E, X705F, XE16o, XE1DG, XE1DG, Xa0hF, Xa0hF, Xa9BV


,read_category,read_code,ICD10,ICD9,description(s)
0,v2,F3967,M350D G737A,3596A 7102D,Myopathy due to Sjogren's disease
1,v2,F4A32,M350D H193A,<NA>,Keratoconjunctivitis sicca (excluding Sjogren'...
2,v2,H57y3,M350D J991A,5178A 7102D,Lung disease with Sjogren's disease
3,v2,N002.,M350,7102,Sicca (Sjogren's) syndrome; Keratoconjunctivit...
0,v3,1927.,M350,<NA>,Dry mouth; Xerostomia
1,v3,F3967,M350D,7102D,Myopathy due to Sjogren's disease
2,v3,F3967,M350D,7102D,Myopathy due to Sjogren's disease
3,v3,F4A32,M350D,<NA>,Keratoconjunctivitis sicca (excluding Sjogren'...
4,v3,F4A32,M350D,<NA>,Keratoconjunctivitis sicca (excluding Sjogren'...
5,v3,H57y3,M350D,7102,Lung disease with Sjogren's disease
